# CHAPER 5 MDP를 모를 때 밸류 평가하기

In [1]:
# 라이브러리 
import random

### 몬테카를로 학습 구현

In [2]:
# Grid World 클래스

class GridWorld():
    def __init__(self):
        self.x=0
        self.y=0
    
    def step(self, a):
        '''
        에이전트로부터 액션을 받아서 상태 변이를 일으키고, 보상을 정해주는 함수
        '''
        if a==0:
            self.move_right()
        elif a==1:
            self.move_left()
        elif a==2:
            self.move_up()
        elif a==3:
            self.move_down()
            
        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done
    
    def move_right(self):
        self.y += 1
        if self.y > 3:
            self.y = 3
        
    def move_left(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0
        
    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0
        
    def move_down(self):
        self.y += 1
        if self.x > 3:
            self.x = 3
    
    def is_done(self):
        '''
        에피소드가 끝났는지 판별해주는 함수
        '''
        if self.x == 3 and self.y == 3:
            return True
        else :
            return False
        
    def get_state(self):
        return (self.x, self.y)
    
    def reset(self):
        '''
        에이전트가 종료 상태에 도달했으면 다시 처름 상태로 돌려놓는 함수
        '''
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [3]:
# Agent 클래스

class Agent():
    def __init__(self):
        pass
    
    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

In [4]:
# 메인 함수

def main():
    env = GridWorld()
    agent = Agent()
    data = [[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]] # 테이블 초기화
    gamma = 1.0 # 감쇄 인자
    alpha = 0.0001 # 업데이트에 사용되는 파라미터
    
    for k in range(1): # 총 n번의 에피소드 진행
        done = False
        history = []
        while not done: # 랜덤 에잌전트가 경험을 쌓는 과정
            action = agent.select_action()
            (x,y), reward, done = env.step(action)
            history.append((x,y,reward))
        env.reset()
            
        # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        for transition in history[::-1]:
            # 방문했던 상태들을 뒤애서부터 보며 차례차례 리턴을 계산
            x, y, reward = transition
            data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])
            cum_reward = cum_reward + gamma+reward
            
    # 학습이 끝나고 난 후 데이터를 출력해보기 위한 코드
    for row in data:
        print(row)

In [5]:
# main()

### Temporal Difference 학습 구현

In [6]:
# 메인 함수

def main():
    env = GridWorld()
    agent = Agent()
    data = [[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]] # 테이블 초기화
    gamma = 1.0 # 감쇄 인자
    alpha = 0.01 # MC에 비해 큰 값을 사용
    
    for k in range(1): # 총 n번의 에피소드 진행
        done = False
        while not done: # 랜덤 에잌전트가 경험을 쌓는 과정
            x, y = env.get_state()
            action = agent.select_action()
            (x_prime,y_prime), reward, done = env.step(action)
            x_prime, y_prime = env.get_state()
            
            # 한 번의 step이 진행되자 말자 바로 테이블의 데이터를 업데이트 해줌
            data[x][y] = data[x][y] + alpha*(reward+gamma*data[x_prime][y_prime]-data[x][y])
        env.reset()
            
    # 학습이 끝나고 난 후 데이터를 출력해보기 위한 코드
    for row in data:
        print(row)

In [ ]:
# main()